In [1]:
import requests
import pandas as pd
import os
import dotenv
import json

dotenv.load_dotenv()

year = 2024
league_id = os.getenv('league_id')
espn_s2 = os.getenv('espn_s2')
swid = os.getenv('swid')

/Users/michaelsilberling/opt/anaconda3/envs/py10/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
cookies = {
    'espn_s2': espn_s2,
    'SWID': swid
}

# Updated API endpoint
url = f"https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{league_id}"

params = {
    "view": ["mMatchup", "mMatchupScore"]
}

headers = {
    "X-Fantasy-Filter": json.dumps({"schedule": {"filterMatchupPeriodIds": {"value": list(range(1, 18))}}})
}

In [3]:
try:
    response = requests.get(url, cookies=cookies, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    print(f"Response content: {response.content}")
    exit(1)

In [4]:
# Fetch team information
team_info_url = f"{url}?view=mTeam"
try:
    team_response = requests.get(team_info_url, cookies=cookies)
    team_response.raise_for_status()
    teams_data = team_response.json()
except requests.exceptions.RequestException as e:
    print(f"Error fetching team data: {e}")
    print(f"Response content: {team_response.content}")
    exit(1)

teams = {team['id']: {'name': team.get('name', '')} for team in teams_data.get('teams', [])}

In [5]:
detailed_data = []

for matchup in data.get('schedule', []):
    week_num = matchup.get('matchupPeriodId')
    for team_type in ['home', 'away']:
        team = matchup.get(team_type)
        if team:
            team_id = team.get('teamId')
            score = team.get('totalPoints')
            team_name = teams.get(team_id, {}).get('name', 'Unknown Team')

            detailed_data.append({
                'Week': week_num,
                'Team ID': team_id,
                'Score': score,
                'Team Name': team_name,
            })

In [6]:
df = pd.DataFrame(detailed_data)

highest_scores_per_week = df.sort_values('Score', ascending=False).drop_duplicates(['Week'])
highest_scores_per_week = highest_scores_per_week.sort_values('Week', ascending=True)

hs_df = highest_scores_per_week[['Week', 'Team Name', 'Score']].set_index('Week')
hs_df.to_csv(f'outputs/weekly_high_scores_{year}.csv')

print(hs_df)

                        Team Name   Score
Week                                     
1                     Cobra Kyler  133.78
2                       EL Diablo  160.44
3               Bloodbath Mcgrath  141.28
4                   Naturally Lit  136.10
5                     BEC Michael  149.32
6                    BDE Culture   144.06
7                     BEC Michael  161.74
8                     BEC Michael  165.36
9                    BDE Culture   156.06
10                    BEC Michael  132.20
11                    Taylor Made  139.94
12              Bloodbath Mcgrath  144.50
13                    BEC Michael  137.78
14    Billy Napier's Fantasy Team  151.34
15                    BEC Michael  172.40
16                   BDE Culture   144.82
17                    BEC Michael    0.00
